<a href="https://colab.research.google.com/github/sandeepgundeboina/LearningSpark/blob/main/SparkExplode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Explode').getOrCreate()

In [3]:
arrayData = [
        ('James',['fridge','washing','Tv','AC']),
        ('Jolly',['fridge','Cooler','Tv',None]),
        ('Julie',[None]),
        ('Rox',['Fridge','Tv',None])]
df=spark.createDataFrame(data=arrayData,schema=['name','appliance'])
df.printSchema()
df.show()


root
 |-- name: string (nullable = true)
 |-- appliance: array (nullable = true)
 |    |-- element: string (containsNull = true)

+-----+--------------------+
| name|           appliance|
+-----+--------------------+
|James|[fridge, washing,...|
|Jolly|[fridge, Cooler, ...|
|Julie|              [NULL]|
|  Rox|  [Fridge, Tv, NULL]|
+-----+--------------------+



In [4]:
arrayBrand = [
        ('James',{'fridge':'LG','washing':'Samsunf','Tv':'Sony','AC':'LG'}),
        ('Jolly',{'fridge':'Haier','Cooler':'Symphony','Tv':''}),
        ('Julie',None),
        ('Rox',{'Fridge':'Bosch','Tv':'LG'})]
df_br=spark.createDataFrame(data=arrayBrand,schema=['name','Brand'])
df_br.printSchema()
df_br.show()

root
 |-- name: string (nullable = true)
 |-- Brand: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)

+-----+--------------------+
| name|               Brand|
+-----+--------------------+
|James|{AC -> LG, Tv -> ...|
|Jolly|{Tv -> , Cooler -...|
|Julie|                NULL|
|  Rox|{Tv -> LG, Fridge...|
+-----+--------------------+



In [5]:
from pyspark.sql.functions import *

In [6]:
df2=df.select(df.name,explode(df.appliance))
df2.show()
df2.printSchema()

+-----+-------+
| name|    col|
+-----+-------+
|James| fridge|
|James|washing|
|James|     Tv|
|James|     AC|
|Jolly| fridge|
|Jolly| Cooler|
|Jolly|     Tv|
|Jolly|   NULL|
|Julie|   NULL|
|  Rox| Fridge|
|  Rox|     Tv|
|  Rox|   NULL|
+-----+-------+

root
 |-- name: string (nullable = true)
 |-- col: string (nullable = true)



    Basic Explode

In [7]:
df2=df_br.select(df_br.name,explode(df_br.Brand,))
df2.show()
df2.printSchema()

+-----+-------+--------+
| name|    key|   value|
+-----+-------+--------+
|James|     AC|      LG|
|James|     Tv|    Sony|
|James|washing| Samsunf|
|James| fridge|      LG|
|Jolly|     Tv|        |
|Jolly| Cooler|Symphony|
|Jolly| fridge|   Haier|
|  Rox|     Tv|      LG|
|  Rox| Fridge|   Bosch|
+-----+-------+--------+

root
 |-- name: string (nullable = true)
 |-- key: string (nullable = false)
 |-- value: string (nullable = true)



    Explode_outer

In [8]:
df2=df_br.select(df_br.name,explode_outer(df_br.Brand,))
df2.show()
df2.printSchema()

+-----+-------+--------+
| name|    key|   value|
+-----+-------+--------+
|James|     AC|      LG|
|James|     Tv|    Sony|
|James|washing| Samsunf|
|James| fridge|      LG|
|Jolly|     Tv|        |
|Jolly| Cooler|Symphony|
|Jolly| fridge|   Haier|
|Julie|   NULL|    NULL|
|  Rox|     Tv|      LG|
|  Rox| Fridge|   Bosch|
+-----+-------+--------+

root
 |-- name: string (nullable = true)
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



    Positional explode

In [9]:
df2=df.select(df.name,posexplode(df.appliance))
df2.show()
df2.printSchema()

+-----+---+-------+
| name|pos|    col|
+-----+---+-------+
|James|  0| fridge|
|James|  1|washing|
|James|  2|     Tv|
|James|  3|     AC|
|Jolly|  0| fridge|
|Jolly|  1| Cooler|
|Jolly|  2|     Tv|
|Jolly|  3|   NULL|
|Julie|  0|   NULL|
|  Rox|  0| Fridge|
|  Rox|  1|     Tv|
|  Rox|  2|   NULL|
+-----+---+-------+

root
 |-- name: string (nullable = true)
 |-- pos: integer (nullable = false)
 |-- col: string (nullable = true)



In [10]:
df2=df_br.select(df_br.name,posexplode(df_br.Brand))
df2.show()
df2.printSchema()

+-----+---+-------+--------+
| name|pos|    key|   value|
+-----+---+-------+--------+
|James|  0|     AC|      LG|
|James|  1|     Tv|    Sony|
|James|  2|washing| Samsunf|
|James|  3| fridge|      LG|
|Jolly|  0|     Tv|        |
|Jolly|  1| Cooler|Symphony|
|Jolly|  2| fridge|   Haier|
|  Rox|  0|     Tv|      LG|
|  Rox|  1| Fridge|   Bosch|
+-----+---+-------+--------+

root
 |-- name: string (nullable = true)
 |-- pos: integer (nullable = false)
 |-- key: string (nullable = false)
 |-- value: string (nullable = true)



    PosExplode_oiuter

In [11]:
df2=df_br.select(df_br.name,posexplode_outer(df_br.Brand))
df2.show()
df2.printSchema()

+-----+----+-------+--------+
| name| pos|    key|   value|
+-----+----+-------+--------+
|James|   0|     AC|      LG|
|James|   1|     Tv|    Sony|
|James|   2|washing| Samsunf|
|James|   3| fridge|      LG|
|Jolly|   0|     Tv|        |
|Jolly|   1| Cooler|Symphony|
|Jolly|   2| fridge|   Haier|
|Julie|NULL|   NULL|    NULL|
|  Rox|   0|     Tv|      LG|
|  Rox|   1| Fridge|   Bosch|
+-----+----+-------+--------+

root
 |-- name: string (nullable = true)
 |-- pos: integer (nullable = true)
 |-- key: string (nullable = true)
 |-- value: string (nullable = true)



    End of Explode FUnction